In [3]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
import numpy as np

In [4]:
from typing import List
from typing import Dict

In [5]:
# model = ResNet50(weights='imagenet')
# tf.keras.utils.plot_model(model, "resnet.png")

In [6]:
model = InceptionResNetV2(weights='imagenet')
tf.keras.utils.plot_model(model, "inception.png")

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.588943 to fit



In [7]:
def get_previous(name):
    inbound = model.get_layer(name).inbound_nodes[0].inbound_layers
    if type(inbound) != list:
        inbound = [inbound]
    return [layer.name for layer in inbound]

def traverse(name):
    # On subsequent recursive steps, the new input layer will be defined, 
    # so that name needs to be checked in base case
    if (name == start_layer) or (name == part_name):
        print("Base case at", name)
        return tf.keras.Input(tensor=model.get_layer(start_layer).output, name=part_name)

    output = []
    for n in get_previous(name):
        output.append(traverse(n))
    
    # If the DAG node only has 1 previous connection
    if len(output) == 1:
        output = output[0]
    
    layer = model.get_layer(name)
    to_next = layer(output)
    return to_next

start_layer = "pool1_pad"
end_layer = "conv2_block1_add"
part_name = "pool1_pad" 
# output = traverse(end_layer)
# part_model = tf.keras.Model(inputs=model.get_layer(start_layer).output, outputs=output)
# tf.keras.utils.plot_model(part_model, "dag_part1.png")

In [8]:
def keras_model_memory_usage_in_bytes(model, batch_size: int):
    """
    Return the estimated memory usage of a given Keras model in bytes.
    This includes the model weights and layers, but excludes the dataset.

    The model shapes are multipled by the batch size, but the weights are not.

    Args:
        model: A Keras model.
        batch_size: The batch size you intend to run the model with. If you
            have already specified the batch size in the model itself, then
            pass `1` as the argument here.
    Returns:
        An estimate of the Keras model's memory usage in bytes.

    """
    default_dtype = tf.keras.backend.floatx()
    shapes_mem_count = 0
    internal_model_mem_count = 0
    for layer in model.layers:
        if isinstance(layer, tf.keras.Model):
            internal_model_mem_count += keras_model_memory_usage_in_bytes(
                layer, batch_size=batch_size
            )
        single_layer_mem = tf.as_dtype(layer.dtype or default_dtype).size
        out_shape = layer.output_shape
        if isinstance(out_shape, list):
            out_shape = out_shape[0]
        for s in out_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = sum(
        [tf.keras.backend.count_params(p) for p in model.trainable_weights]
    )
    non_trainable_count = sum(
        [tf.keras.backend.count_params(p) for p in model.non_trainable_weights]
    )

    total_memory = (
        batch_size * shapes_mem_count
        + internal_model_mem_count
        + trainable_count
        + non_trainable_count
    )
    return total_memory

In [9]:
def keras_layer_memory(layer, batch_size: int):
    default_dtype = tf.keras.backend.floatx()
    shapes_mem_count = 0
    internal_model_mem_count = 0
    
    if isinstance(layer, tf.keras.Model):
        internal_model_mem_count += keras_model_memory_usage_in_bytes(
            layer, batch_size=batch_size
        )
    single_layer_mem = tf.as_dtype(layer.dtype or default_dtype).size
    out_shape = layer.output_shape
    if isinstance(out_shape, list):
        out_shape = out_shape[0]
    for s in out_shape:
        if s is None:
            continue
        single_layer_mem *= s
    shapes_mem_count += single_layer_mem

    trainable_count = sum(
        [tf.keras.backend.count_params(p) for p in layer.trainable_weights]
    )
    non_trainable_count = sum(
        [tf.keras.backend.count_params(p) for p in layer.non_trainable_weights]
    )

    total_memory = (
        batch_size * shapes_mem_count
        + internal_model_mem_count
        + trainable_count
        + non_trainable_count
    )
    return total_memory

In [10]:
layer_mems = {}
for layer in model.layers:
    mem = keras_layer_memory(layer, 1)
    layer_mems[layer.name] = mem

print(layer_mems)

{'input_1': 1072812, 'conv2d': 2842592, 'batch_normalization': 2841824, 'activation': 2841728, 'conv2d_1': 2775168, 'batch_normalization_1': 2766048, 'activation_1': 2765952, 'conv2d_2': 5550336, 'batch_normalization_2': 5532096, 'activation_2': 5531904, 'max_pooling2d': 1364224, 'conv2d_3': 1710400, 'batch_normalization_3': 1705520, 'activation_3': 1705280, 'conv2d_4': 4009728, 'batch_normalization_4': 3872064, 'activation_4': 3871488, 'max_pooling2d_1': 940800, 'conv2d_8': 325888, 'batch_normalization_8': 313792, 'activation_8': 313600, 'conv2d_6': 244416, 'conv2d_9': 525696, 'batch_normalization_6': 235344, 'batch_normalization_9': 470688, 'activation_6': 235200, 'activation_9': 470400, 'average_pooling2d': 940800, 'conv2d_5': 488832, 'conv2d_7': 390400, 'conv2d_10': 553344, 'conv2d_11': 325888, 'batch_normalization_5': 470688, 'batch_normalization_7': 313792, 'batch_normalization_10': 470688, 'batch_normalization_11': 313792, 'activation_5': 470400, 'activation_7': 313600, 'activat

In [11]:
def get_next(model, name):
    outbound = model.get_layer(name).outbound_nodes
    return [node.outbound_layer.name for node in outbound]

In [12]:
# A recursive function used by longestPath. See below
# link for details
# https:#www.geeksforgeeks.org/topological-sorting/
def topologicalSortUtil(v: str):
	global Stack, visited
	visited[v] = True

	# Recur for all the vertices adjacent to this vertex
	# list<AdjListNode>::iterator i
	for i in get_next(model, v):
		if (not visited[i]):
			topologicalSortUtil(i)

	# Push current vertex to stack which stores topological
	# sort
	Stack.append(v)

# The function to find longest distances from a given vertex.
# It uses recursive topologicalSortUtil() to get topological
# sorting.
def longestPath(s: str):
	global Stack, dist, visited
	for l in model.layers:
		visited[l.name] = False
		dist[l.name] = -1 # Equal to -infty

	# Call the recursive helper function to store Topological
	# Sort starting from all vertices one by one
	for l in model.layers:
		if (visited[l.name] == False):
			topologicalSortUtil(l.name)

	# Initialize distances to all vertices as infinite and
	# distance to source as 0
	dist[s] = 0
	# Stack.append(1)

	# Process vertices in topological order
	while (len(Stack) > 0):

		# Get the next vertex from topological order
		u = Stack.pop()

		# Update distances of all adjacent vertices
		# list<AdjListNode>::iterator i
		if (dist[u] != -1):
			for i in get_next(model, u):
				if (dist[i] < dist[u] + 1):
					dist[i] = dist[u] + 1 # Each edge weighted 1

	# Print calculated longest distances
	levels = [[]] * len(dist)
	for l in model.layers:
		if len(levels[dist[l.name]]) == 0:
			levels[dist[l.name]] = []
		
		levels[dist[l.name]].append(l.name)
	
	return levels


Stack = []
visited = {}
dist = {}

name = model.input._keras_history.layer.name
levels = longestPath(name)
singles = []
for l in range(len(levels)):
	if len(levels[l]) == 1:
		singles.append(levels[l][0])

print(singles)

['input_1', 'conv2d', 'batch_normalization', 'activation', 'conv2d_1', 'batch_normalization_1', 'activation_1', 'conv2d_2', 'batch_normalization_2', 'activation_2', 'max_pooling2d', 'conv2d_3', 'batch_normalization_3', 'activation_3', 'conv2d_4', 'batch_normalization_4', 'activation_4', 'max_pooling2d_1', 'conv2d_10', 'batch_normalization_10', 'activation_10', 'mixed_5b', 'conv2d_17', 'batch_normalization_17', 'activation_17', 'block35_1_mixed', 'block35_1_conv', 'block35_1', 'block35_1_ac', 'conv2d_23', 'batch_normalization_23', 'activation_23', 'block35_2_mixed', 'block35_2_conv', 'block35_2', 'block35_2_ac', 'conv2d_29', 'batch_normalization_29', 'activation_29', 'block35_3_mixed', 'block35_3_conv', 'block35_3', 'block35_3_ac', 'conv2d_35', 'batch_normalization_35', 'activation_35', 'block35_4_mixed', 'block35_4_conv', 'block35_4', 'block35_4_ac', 'conv2d_41', 'batch_normalization_41', 'activation_41', 'block35_5_mixed', 'block35_5_conv', 'block35_5', 'block35_5_ac', 'conv2d_47', 'b

In [13]:
def findAllPathsUtil(u, d, visited, path, all_paths):
    # Mark the current node as visited and store in path
    if dist[u] > dist[d]:
        return False
    visited[u] = True
    path.append(u)

    # If current vertex is same as destination, then print
    # current path[]
    if u == d:
        exists = False
        # See if path already exists in list of paths
        for p in all_paths:
            if p == path:
                exists = True
                break
        
        if exists == False:
            all_paths.append(path.copy())
    else:
        # If current vertex is not destination
        # Recur for all the vertices adjacent to this vertex
        for i in get_next(model, u):
            if visited[i] == False:
                ret = findAllPathsUtil(i, d, visited, path, all_paths)
                if ret == False:
                    return False
                
    # Remove current vertex from path[] and mark it as unvisited
    path.pop()
    visited[u] = False
    return True


# Prints all paths from 's' to 'd'
def findAllPaths(s, d):
    # Mark all the vertices as not visited
    visited = {}
    for l in model.layers:
        visited[l.name] = False

    # Create an array to store paths
    path = []
    all_paths = []

    # Call the recursive helper function to print all paths
    return findAllPathsUtil(s, d, visited, path, all_paths)

In [14]:
def partitionsUtil(prev, nodes, parts):
    if len(nodes) == 0:
        return parts
    p = False
    i = -1 # So first i starts at 0
    while p == False:
        i += 1
        p = findAllPaths(prev, nodes[i])
    
    parts.append(nodes[i])
    return partitionsUtil(nodes[i], nodes[i+1:], parts)

def find_partitions():
    inpt = model.input._keras_history.layer.name
    return partitionsUtil(inpt, singles, [])

parts = find_partitions()

In [15]:
def findPartitionMemory(parts):
    parts_to_mem = {}
    parts_mem = []
    for i in range(1, len(parts)):
        # Going backwards along layers to find total memory usage
        start = dist[parts[i]]
        end = dist[parts[i-1]]
        mem = 0
        for j in range(start, end, -1):
            for l in levels[j]:
                layer_mem = keras_layer_memory(model.get_layer(l), 1)
                mem += layer_mem
        parts_to_mem[parts[i]] = mem
        parts_mem.append(mem)
    return parts_to_mem, parts_mem

partition_mem_dict, partition_mems = findPartitionMemory(parts)
print(partition_mem_dict)
print(len(partition_mems))


{'conv2d': 2842592, 'batch_normalization': 2841824, 'activation': 2841728, 'conv2d_1': 2775168, 'batch_normalization_1': 2766048, 'activation_1': 2765952, 'conv2d_2': 5550336, 'batch_normalization_2': 5532096, 'activation_2': 5531904, 'max_pooling2d': 1364224, 'conv2d_3': 1710400, 'batch_normalization_3': 1705520, 'activation_3': 1705280, 'conv2d_4': 4009728, 'batch_normalization_4': 3872064, 'activation_4': 3871488, 'max_pooling2d_1': 940800, 'mixed_5b': 10539248, 'block35_1': 7414608, 'block35_1_ac': 1568000, 'block35_2': 7414608, 'block35_2_ac': 1568000, 'block35_3': 7414608, 'block35_3_ac': 1568000, 'block35_4': 7414608, 'block35_4_ac': 1568000, 'block35_5': 7414608, 'block35_5_ac': 1568000, 'block35_6': 7414608, 'block35_6_ac': 1568000, 'block35_7': 7414608, 'block35_7_ac': 1568000, 'block35_8': 7414608, 'block35_8_ac': 1568000, 'block35_9': 7414608, 'block35_9_ac': 1568000, 'block35_10': 7414608, 'block35_10_ac': 1568000, 'mixed_6a': 14483712, 'block17_1': 6417312, 'block17_1_ac'

In [16]:
# Returns 0 or 1 depending on whether sum1 or sum2 was bigger,
# and returns the max of them
def argmax(sum1, sum2):
	mx = 0
	mx_ind = -1
	if sum1 >= sum2:
		mx = sum1
		mx_ind = 0
	else:
		mx = sum2
		mx_ind = 1
	return mx, mx_ind

# Python3 program to find subarray having
# maximum sum less than or equal to sum

# To find subarray with maximum sum
# less than or equal to sum
def findMaxSubarraySum(arr, n, sum):
	
	# To store current sum and
	# max sum of subarrays
	curr_sum = arr[0]
	start = 0
	end = 0

	max_sum = 0
	max_start = 0
	max_end = 0

	# To find max_sum less than sum
	for i in range(1, n):
		
		# Update max_sum if it becomes
		# greater than curr_sum
		if (curr_sum <= sum):
			max_sum, ind = argmax(max_sum, curr_sum)
			if ind == 1:
				max_start, max_end = start, end

		# If curr_sum becomes greater than sum
		# subtract starting elements of array
		while (curr_sum + arr[i] > sum and start < i):
			curr_sum -= arr[start]
			start += 1
		
		# Add elements to curr_sum
		curr_sum += arr[i]
		end = i

	# Adding an extra check for last subarray
	if (curr_sum <= sum):
		max_sum, ind = argmax(max_sum, curr_sum)
		if ind == 1:
				max_start, max_end = start, end

	return max_sum, max_start, max_end

bound = 100 * (1024 ** 2)
part_size, part_start, part_end = findMaxSubarraySum(partition_mems, len(partition_mems), bound)
print(parts[part_start:part_end])


['block17_10', 'block17_10_ac', 'block17_11', 'block17_11_ac', 'block17_12', 'block17_12_ac', 'block17_13', 'block17_13_ac', 'block17_14', 'block17_14_ac', 'block17_15', 'block17_15_ac', 'block17_16', 'block17_16_ac', 'block17_17', 'block17_17_ac', 'block17_18', 'block17_18_ac', 'block17_19', 'block17_19_ac', 'block17_20', 'block17_20_ac', 'mixed_7a', 'block8_1', 'block8_1_ac', 'block8_2', 'block8_2_ac', 'block8_3', 'block8_3_ac']


In [20]:
def partitionModel(part_mems: List[int], parts: List[str], caps, models: List[List[List[str]]]):
    if len(part_mems) == 0:
        return
    # Find sort capacities to find top 2
    by_size_inds = np.argsort(caps)
    # Max partition that fits into biggest 2 physical nodes
    max_cap_ind = by_size_inds[-2]

    part_size, part_start, part_end = findMaxSubarraySum(part_mems, len(part_mems), caps[max_cap_ind])
    #print(part_size)

    # First try to place partition in max capacity node
    i = -1
    # Looping until it can find a node that can hold the partition
    while caps[by_size_inds[-i]] < sum(part_mems[part_start:part_end+1]):
        i += 1
        # Couldn't find place to put partition
        if i >= len(by_size_inds):
            return
    models[by_size_inds[-i]].append(parts[part_start:part_end+1])
    caps[by_size_inds[-i]] -= part_size
    #print(caps)
    leftover_mem = [part_mems[:part_start], part_mems[part_end+1:]]
    leftover_parts = [parts[:part_start], parts[part_end+1:]]
    for l in range(len(leftover_mem)):
        # Recurse on leftover capacity and available partitions
        partitionModel(leftover_mem[l], leftover_parts[l], caps, models)

nodes = [100 * (1024 ** 2), 50 * (1024 ** 2), 100 * (1024 ** 2), 100 * (1024 ** 2)]
caps = nodes.copy()
# Modified by reference in the function
models = []
for i in range(len(nodes)):
    models.append([])
partitionModel(partition_mems, parts, caps, models)
print(models)
count = 0
for i in range(len(models)):
    #print(f"Node {i+1}: {len(models[i])}")
    for j in range(len(models[i])):
        #print(f"Partition Length: {len(models[i][j])}")
        count += len(models[i][j])

# Still case 1 right now, doesn't deal with case of total model size being 
# greater than physical node capacity yet
print(f'{count} out of {len(partition_mems)}')
for n in range(len(nodes)):
    print(f"Node {n+1}: {caps[n]}/{nodes[n]}")

[[['block17_10', 'block17_10_ac', 'block17_11', 'block17_11_ac', 'block17_12', 'block17_12_ac', 'block17_13', 'block17_13_ac', 'block17_14', 'block17_14_ac', 'block17_15', 'block17_15_ac', 'block17_16', 'block17_16_ac', 'block17_17', 'block17_17_ac', 'block17_18', 'block17_18_ac', 'block17_19', 'block17_19_ac', 'block17_20', 'block17_20_ac', 'mixed_7a', 'block8_1', 'block8_1_ac', 'block8_2', 'block8_2_ac', 'block8_3', 'block8_3_ac', 'block8_4']], [['mixed_5b'], ['block17_2', 'block17_2_ac', 'block17_3', 'block17_3_ac', 'block17_4', 'block17_4_ac', 'block17_5', 'block17_5_ac', 'block17_6', 'block17_6_ac', 'block17_7']], [['conv2d_1', 'batch_normalization_1', 'activation_1', 'conv2d_2', 'batch_normalization_2', 'activation_2', 'max_pooling2d', 'conv2d_3', 'batch_normalization_3', 'activation_3', 'conv2d_4', 'batch_normalization_4', 'activation_4', 'max_pooling2d_1'], ['input_1', 'conv2d', 'batch_normalization', 'activation'], ['block17_1_ac'], ['block17_8'], ['block17_7_ac'], ['block17_9

In [18]:
# Python program to find number of bins required using
# First Fit algorithm.

# Returns number of bins required using first fit
# online algorithm
def firstFit(weight, n, c):
	
	# Initialize result (Count of bins)
	res = 0
	
	# Create an array to store remaining space in bins
	# there can be at most n bins
	bin_rem = [0]*n
	
	# Place items one by one
	for i in range(n):
	
		# Find the first bin that can accommodate
		# weight[i]
		j = 0
		while( j < res):
			if (bin_rem[j] >= weight[i]):
				bin_rem[j] = bin_rem[j] - weight[i]
				break
			j+=1
			
		# If no bin could accommodate weight[i]
		if (j == res):
			bin_rem[res] = c - weight[i]
			res= res+1
	return res
	
# Driver program
weight = [2, 5, 4, 7, 1, 3, 8]
c = 10
n = len(weight)
print("Number of bins required in First Fit : ",firstFit(weight, n, c))

Number of bins required in First Fit :  4


In [23]:
def get_flops():
    session = tf.compat.v1.Session()
    graph = tf.compat.v1.get_default_graph()

    with graph.as_default():
        with session.as_default():
            model = InceptionResNetV2(weights='imagenet')
            run_meta = tf.compat.v1.RunMetadata()
            opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

            # We use the Keras session graph in the call to the profiler.
            flops = tf.compat.v1.profiler.profile(graph=graph,
                                                  run_meta=run_meta, cmd='op', options=opts)

    tf.compat.v1.reset_default_graph()

    return flops.total_float_ops

print(get_flops())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
55857901
